In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
import requests
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

THIẾT LẬP CHỐNG CHẶN BOT CỦA WEBSITE

In [ ]:
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

ĐỌC FILE CSV ĐỂ LẤY RA CÁC URL BÀI VIẾT

In [ ]:
df = pd.read_csv('urls_all_page_thang_6_1_100.csv')
urls = df['URL'].tolist()

XỬ LÝ LẤY PAGE_SOURCE CỦA POST

SELLER INFOMATION

In [ ]:
def Sales_Info(page_source):
    seller_info = page_source.find('div', class_ = 'SellerInfo_sellerWrapper__r4S9i')

    seller_name = seller_info.find('div', class_ = 'SellerInfo_nameDiv__rWqQB').text.strip()
    seller_role = seller_info.find('div', class_ = 'd4deiyo').text.strip()

    # print('Người bán: ', seller_name)
    # print('Loại đối tượng: ', seller_role)

    return seller_name, seller_role

THÔNG TIN CHUNG VỀ BẤT ĐỘNG SẢN

In [ ]:
def RealEstate_Info(page_source, driver):
    RealEstate_Info = page_source.find('div', class_ = 'cd9gm5n')

    details = driver.find_elements(By.CLASS_NAME, 'brnpcl3')

    items = driver.find_elements(By.CSS_SELECTOR, 'div.i1qen30x.r9vw5if')

    phong_ngu = 'Không rõ'
    huong = 'Không rõ'
    loai_nha = 'Không rõ'
    for item in items:
        try:
            lines = item.text.strip().split('\n')  # Tách từng dòng riêng biệt
            for line in lines:
                line = line.strip()
                if 'PN' in line:
                    phong_ngu = line  # Không cần lấy lại từ <span>, vì line đã là nội dung chuẩn
                elif 'Hướng' in line:
                    huong = line
                elif 'Nhà' in line or 'Chung cư' in line or 'Căn hộ' in line or 'Penhouse' in line or 'Đất' in line or 'Tập thể' in line:
                    loai_nha = line
        except Exception as e:
            continue



    RE_Money = RealEstate_Info.find('b', class_ = 'pyhk1dv').text.strip()
    RE_Money_Per_SqM = details[0].text.strip()  # Giá tiền trên m2
    RE_Area = details[1].text.strip()   # Diện tích đất

    RE_Location = RealEstate_Info.find('span', class_ = 'bwq0cbs flex-1').text.strip()

    return phong_ngu, huong, loai_nha, RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location


THÔNG TIN CHI TIẾT CỦA BẤT ĐỘNG SẢN

In [ ]:
def RealEstate_Detail(driver):
    # Lấy tất cả các thẻ div thông tin
    items = driver.find_elements(By.CSS_SELECTOR, "div.col-xs-6.abzctes")

    giay_to_phap_ly = "Không rõ"
    so_phong_ve_sinh = "Không rõ"
    tinh_trang_noi_that = "Không rõ"
    so_tang = "Không rõ"

    for item in items:
        try:
            label = item.text.strip()
            if "Giấy tờ pháp lý" in label:
                # Tìm strong hoặc thẻ chứa giá trị trong cùng item
                value_el = item.find_element(By.TAG_NAME, "strong")
                giay_to_phap_ly = value_el.text.strip()
            elif "Số phòng vệ sinh" in label:
                value_el = item.find_element(By.TAG_NAME, "strong")
                so_phong_ve_sinh = value_el.text.strip()
            elif "Tổng số tầng" in label:
                value_el = item.find_element(By.TAG_NAME, "strong")
                so_tang = value_el.text.strip()
            elif "Tình trạng nội thất" in label:
                value_el = item.find_element(By.TAG_NAME, "strong")
                tinh_trang_noi_that = value_el.text.strip()
        except:
            continue

    return giay_to_phap_ly, tinh_trang_noi_that, so_tang, so_phong_ve_sinh


NỐI CÁC HÀM LẠI VÀO VÒNG FOR VÀ LƯU RA FILE CSV.

In [ ]:
driver = uc.Chrome(options=options)
actions = ActionChains(driver)
sleep(1)

results = []

for i, url in enumerate(urls):
    print(f"\nĐang xử lý: {url}")
    driver.get(url)
    sleep(2)

    # Click vào vị trí (0, 0) hoặc giữa màn hình
    if i == 0:
        try:
            actions.move_by_offset(10, 10).click().perform()
            sleep(3)
        except Exception as e:
            print(f"Không thể click offset: {e}")

    # Click vào nút "Xem thêm" để tải thêm nội dung và chờ 5 giây để đầy đủ nội dung
    try:
        more_field = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[4]/div[1]/div/div[4]/div/div[2]/div/div[2]/div[2]/button')
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", more_field)
        sleep(2)
        more_field.click()
        sleep(4)
    except Exception as e:
        print(f"Không tìm thấy hoặc không click được nút 'Xem thêm': {e}")

    page_source = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        seller_name, seller_role = Sales_Info(page_source)
    except:
        seller_name, seller_role = "Không rõ", "Không rõ"

    try:
        phong_ngu, huong, loai_nha, RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location = RealEstate_Info(page_source, driver)
    except:
        phong_ngu = huong = loai_nha = RE_Money = RE_Money_Per_SqM = RE_Area = RE_Location = "Không rõ"

    try:
        giay_to_phap_ly, so_phong_ve_sinh, tinh_trang_noi_that, so_tang = RealEstate_Detail(driver)
    except:
        giay_to_phap_ly = tinh_trang_noi_that = so_tang = "Không rõ"

    # Thêm dữ liệu vào list
    results.append([
        url,
        seller_name, seller_role,
        phong_ngu, huong, loai_nha,
        RE_Money, RE_Money_Per_SqM, RE_Area, RE_Location,
        giay_to_phap_ly, so_phong_ve_sinh, tinh_trang_noi_that, so_tang
    ])


Đang xử lý: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-hoang-mai-ha-noi/125859741.htm#px=SR-stickyad-[PO-1][PL-top]

Đang xử lý: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-nam-tu-liem-ha-noi/125719373.htm#px=SR-stickyad-[PO-1][PL-top]

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-quan-thanh-xuan-ha-noi/125705844.htm#px=SR-stickyad-[PO-1][PL-top]

Đang xử lý: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-ha-dong-ha-noi/122864659.htm#px=SR-stickyad-[PO-1][PL-top]

Đang xử lý: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-ba-dinh-ha-noi/125719644.htm#px=SR-stickyad-[PO-1][PL-top]

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-huyen-hoai-duc-ha-noi/125874110.htm

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-quan-nam-tu-liem-ha-noi/125824268.htm#px=SR-special_display_ad-[PO-1][PL-default]

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-quan-ha-dong-ha-noi/125654855.htm

Đang xử lý: https://www.nhatot.com/mua-ban-nha-dat-quan-hoang-mai-ha-n

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x763b03+62899]
	GetHandleVerifier [0x0x763b44+62964]
	(No symbol) [0x0x5910f3]
	(No symbol) [0x0x56ff59]
	(No symbol) [0x0x604f7e]
	(No symbol) [0x0x61f6a9]
	(No symbol) [0x0x5fe306]
	(No symbol) [0x0x5cd670]
	(No symbol) [0x0x5ce4e4]
	GetHandleVerifier [0x0x9c4793+2556483]
	GetHandleVerifier [0x0x9bfd02+2537394]
	GetHandleVerifier [0x0x78a2fa+220586]
	GetHandleVerifier [0x0x77aae8+157080]
	GetHandleVerifier [0x0x78141d+184013]
	GetHandleVerifier [0x0x76ba68+95512]
	GetHandleVerifier [0x0x76bc10+95936]
	GetHandleVerifier [0x0x756b5a+9738]
	BaseThreadInitThunk [0x0x7702fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x773582ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x7735827e+238]


In [ ]:
# Lưu dữ liệu ra file CSV
columns = [
    "URL",
    "Người bán", "Loại người bán",
    "Số phòng ngủ", "Hướng", "Kiểu nhà",
    "Giá", "Giá mỗi m2", "Diện tích", "Địa chỉ",
    "Giấy tờ pháp lý", "Số phòng vệ sinh", "Tình trạng nội thất", "Số tầng"
]
df_output = pd.DataFrame(results, columns=columns)
df_output.to_csv('thong_tin_bds_thang_6_1_100.csv', index=False, encoding='utf-8-sig')

print("Đã lưu dữ liệu vào file thong_tin_bds_thang_6_1_100.csv")

PHẦN CHƯA DÙNG TỚI